In [ ]:
import pandas as pd

# Load the CSV file
file_path = "/content/QPSD_ExpandedAtt.csv"
df = pd.read_csv(file_path)

# Strip whitespace from column names
df.columns = df.columns.str.strip()

# Drop unnecessary unnamed columns
df = df.loc[:, ~df.columns.str.contains("Unnamed")]

# Process each year separately
for year in [2021, 2022, 2023]:
    year_cols = [f"{year}Q{i}" for i in range(1, 5) if f"{year}Q{i}" in df.columns]

    if year_cols:  # Ensure the columns exist
        # Identify rows where "Series Name" contains "%"
        percentage_rows = df["Series Name"].str.contains("%", na=False)

        # Sum absolute values
        df.loc[~percentage_rows, year] = df.loc[~percentage_rows, year_cols].apply(pd.to_numeric, errors='coerce').sum(axis=1)

        # Average percentage-based values
        df.loc[percentage_rows, year] = df.loc[percentage_rows, year_cols].apply(pd.to_numeric, errors='coerce').mean(axis=1)

        # Drop the original quarterly columns
        df.drop(columns=year_cols, inplace=True)

# Save the transformed dataset
output_path = "/content/QPSD_Transformed.csv"
df.to_csv(output_path, index=False)

# Display confirmation
print(f"Transformed data saved to: {output_path}")



Transformed data saved to: /content/QPSD_Transformed.csv


In [ ]:
import pandas as pd
import glob

# Define the path where the CSV files are stored (adjust accordingly)
file_paths = ["/content/Economic_Indicators.csv", "/content/QPSD_Transformed.csv", "/content/Statistical_Indicators.csv", "//content/socialIndicatorsAdded.csv", "/content/Environmental_Indicators.csv"]

# List to store transformed dataframes
df_list = []

for file in file_paths:
    # Read each file
    df = pd.read_csv(file)

    # Convert wide format (2021, 2022, 2023) into long format
    df_melted = df.melt(id_vars=["Country Name", "Country Code", "Series Name", "Series Code"],
                         var_name="Year",
                         value_name="Value")

    # Convert "Year" column to numeric type
    df_melted["Year"] = pd.to_numeric(df_melted["Year"])

    # Append to list
    df_list.append(df_melted)

# Concatenate all transformed data
df_final = pd.concat(df_list, ignore_index=True)

# Save the final reshaped dataset
output_path = "/content/reshaped_data.csv"
df_final.to_csv(output_path, index=False)

print(f"Reshaped data saved to: {output_path}")


Reshaped data saved to: /content/reshaped_data.csv
